I am trying to test reinjection for different structures. this is just a general test.

the idea is to remove the acsl in combined file in a new column, and then reinject right after.

combined_file -> code_implementation -> combined_file

In [ ]:
import pandas as pd
import re
import ast
import modal
from validation_treesitter import inject_acsl_specs_treesitter, extract_structure, validate_and_inject

In [67]:
df = pd.read_csv('casp_pt3_1.csv')
df.head()

,file_name,combined_file,acsl_snippet,code_snippet
0,frama_c_files_v2/test_frama_c_2054.c,/*@ assigns \nothing;\n ensures \result == ...,/*@ assigns \nothing;\n ensures \result == ...,"int clamp(int v, int min, int max)\n{\n\tint l..."
1,frama_c_files_v2/test_frama_c_717.c,/*@ requires n > 0;\n assigns \nothing;\n ...,/*@ requires n > 0;\n assigns \nothing;\n ...,int pick_index(int n) { return 0; }
2,frama_c_files_v2/test_frama_c_2473.c,#include <limits.h>\n#include <string.h>\n/*@ ...,#include <limits.h>\n#include <string.h>\n\n/*...,int matcher_ab(char * x0) {\n char x2 = x0[0...
3,frama_c_files_v2/test_frama_c_2371.c,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,"void q2(int a[],int n,int k)\n{\n/*@\n\tloop i..."
4,frama_c_files_v2/test_frama_c_1505.c,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,"void bubbleinnerloop(int a[], int n)\n{\n\t/*@..."


In [ ]:
def remove_all_comments(c_code: str) -> str:
    if pd.isna(c_code) or not c_code:
        return ""
    c_code = re.sub(r'/\*.*?\*/', '', c_code, flags=re.DOTALL)
    c_code = re.sub(r'//.*?$', '', c_code, flags=re.MULTILINE)
    c_code = re.sub(r'\n\s*\n+', '\n\n', c_code)
    
    return c_code.strip()

df['code_implementation'] = df['combined_file'].apply(remove_all_comments)
df.head()

In [ ]:
row = df.iloc[2]
code_impl = row['code_implementation']

print('Initial:\n' + row['combined_file'] + '\n')
print('Reinjected:\n' + inject_acsl_specs_treesitter(ast.literal_eval(row['spec']), code_impl) + '\n')

In [ ]:
def reinject_row(row):
    try:
        spec = ast.literal_eval(row['spec'])
        return inject_acsl_specs_treesitter(spec, row['code_implementation'])
    except Exception as e:
        return f"ERROR: {e}"

df['reinjection'] = df.apply(reinject_row, axis=1)
df.head()

In [ ]:
df.drop(columns=['code_implementation','spec'], inplace=True)

In [ ]:
df.to_csv('compare.csv', index=False)

now want to see if the reinjetion passes

In [ ]:
# Get reference to the deployed Modal function
check_compilation = modal.Function.from_name("comp_check", "check_compilation")

# Spawn all jobs in parallel
reinjection_codes = df['reinjection'].tolist()
handles = [check_compilation.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
results = [h.get() for h in handles]

# Add results as new column
df['compiles'] = results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Compiles: {sum(results)}")
print(f"Fails: {len(results) - sum(results)}")
print(f"Pass rate: {sum(results)/len(results)*100:.1f}%")

# Show failures
failures = df[~df['compiles']]
print(f"\nFailed rows: {len(failures)}")
if len(failures) > 0:
    print(failures.index.tolist()[:20])


In [ ]:
# Combine reinjection with unit tests for runtime testing
df['reinjection_with_tests'] = df['reinjection'] + '\n\n' + df['unit_tests']

# Get reference to the deployed runtime check function
check_runtime = modal.Function.from_name("run_check", "check_runtime")

# Spawn all runtime checks in parallel
test_codes = df['reinjection_with_tests'].tolist()
print(f"Spawning {len(test_codes)} runtime checks...")

handles = [check_runtime.spawn(code) for code in test_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
runtime_results = [h.get() for h in handles]

# Add results as new column
df['tests_pass'] = runtime_results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Tests pass: {sum(runtime_results)}")
print(f"Tests fail: {len(runtime_results) - sum(runtime_results)}")
print(f"Pass rate: {sum(runtime_results)/len(runtime_results)*100:.1f}%")

# Show failures
test_failures = df[~df['tests_pass']]
print(f"\nFailed rows: {len(test_failures)}")
if len(test_failures) > 0:
    print(test_failures.index.tolist()[:20])


In [68]:
# Test reinjection with Frama-C verification
verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

# Spawn all Frama-C verification jobs in parallel
reinjection_codes = df['combined_file'].tolist()
print(f"Spawning {len(reinjection_codes)} Frama-C verification checks...")

handles = [verify_annotated_c.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
frama_results = [h.get() for h in handles]

# Add results as new column
df['frama_verified'] = frama_results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Verified: {sum(frama_results)}")
print(f"Not verified: {len(frama_results) - sum(frama_results)}")
print(f"Pass rate: {sum(frama_results)/len(frama_results)*100:.1f}%")

# Show failures
frama_failures = df[~df['frama_verified']]
print(f"\nFailed rows: {len(frama_failures)}")
if len(frama_failures) > 0:
    print(frama_failures.index.tolist()[:20])

Spawning 359 Frama-C verification checks...
All jobs spawned. Collecting results...

Total: 359
Verified: 348
Not verified: 11
Pass rate: 96.9%

Failed rows: 11
[19, 29, 36, 42, 65, 87, 97, 101, 221, 257, 289]


In [ ]:
new_df = pd.read_csv("casp_pt1.csv")

In [ ]:
# Test reinjection with Frama-C verification
verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

reinjection_codes = new_df['combined_file'].tolist()
print(f"Spawning {len(reinjection_codes)} Frama-C verification checks...")

handles = [verify_annotated_c.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
frama_results = [h.get() for h in handles]

# Add results as new column
new_df['frama_verified'] = frama_results

# Summary
print(f"\nTotal: {len(new_df)}")
print(f"Verified: {sum(frama_results)}")
print(f"Not verified: {len(frama_results) - sum(frama_results)}")
print(f"Pass rate: {sum(frama_results)/len(frama_results)*100:.1f}%")

# Show failures
frama_failures = new_df[~new_df['frama_verified']]
print(f"\nFailed rows: {len(frama_failures)}")
if len(frama_failures) > 0:
    print(frama_failures.index.tolist()[:20])


#### Issues:
When we ran reinjection around 30 failed. Then I check combined_code from casp_pt4_1.csv and 7 failed. Where did these issues come from?
casp_pt1.csv passed -> 

In [59]:
casp_pt3_1 = pd.read_csv("casp_pt3_1.csv")
casp_pt3_1.head()

,file_name,combined_file,acsl_snippet,code_snippet
0,frama_c_files_v2/test_frama_c_2054.c,/*@ assigns \nothing;\n ensures \result == ...,/*@ assigns \nothing;\n ensures \result == ...,"int clamp(int v, int min, int max)\n{\n\tint l..."
1,frama_c_files_v2/test_frama_c_717.c,/*@ requires n > 0;\n assigns \nothing;\n ...,/*@ requires n > 0;\n assigns \nothing;\n ...,int pick_index(int n) { return 0; }
2,frama_c_files_v2/test_frama_c_2473.c,#include <limits.h>\n#include <string.h>\n/*@ ...,#include <limits.h>\n#include <string.h>\n\n/*...,int matcher_ab(char * x0) {\n char x2 = x0[0...
3,frama_c_files_v2/test_frama_c_2371.c,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,"void q2(int a[],int n,int k)\n{\n/*@\n\tloop i..."
4,frama_c_files_v2/test_frama_c_1505.c,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,"void bubbleinnerloop(int a[], int n)\n{\n\t/*@..."


In [62]:
# Test reinjection with Frama-C verification
verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

reinjection_codes = casp_pt3_1['combined_file'].tolist()
print(f"Spawning {len(reinjection_codes)} Frama-C verification checks...")

handles = [verify_annotated_c.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
frama_results = [h.get() for h in handles]

# Add results as new column
casp_pt3_1['frama_verified'] = frama_results

# Summary
print(f"\nTotal: {len(casp_pt3_1)}")
print(f"Verified: {sum(frama_results)}")
print(f"Not verified: {len(frama_results) - sum(frama_results)}")
print(f"Pass rate: {sum(frama_results)/len(frama_results)*100:.1f}%")

# Show failures
frama_failures = casp_pt3_1[~casp_pt3_1['frama_verified']]
print(f"\nFailed rows: {len(frama_failures)}")
if len(frama_failures) > 0:
    print(frama_failures.index.tolist()[:20])


Spawning 359 Frama-C verification checks...
All jobs spawned. Collecting results...

Total: 359
Verified: 349
Not verified: 10
Pass rate: 97.2%

Failed rows: 10
[19, 29, 36, 42, 87, 97, 101, 221, 257, 289]


In [65]:
# Check rows 257 and 101
check_indices = [19, 29, 36, 42, 87, 97, 101, 221, 257, 289]

verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

for idx in check_indices:
    code = df.iloc[idx]['combined_file']
    result = verify_annotated_c.spawn(code).get()
    status = "✓ PASS" if result else "✗ FAIL"
    print(f"Row {idx}: {status}")

Row 19: ✗ FAIL
Row 29: ✗ FAIL
Row 36: ✗ FAIL
Row 42: ✗ FAIL
Row 87: ✓ PASS
Row 97: ✓ PASS
Row 101: ✓ PASS
Row 221: ✓ PASS
Row 257: ✓ PASS
Row 289: ✓ PASS
